Sources

#### Projet End-to-End (Pas NHL API)
https://medium.com/coinmonks/4-718-using-machine-learning-to-bet-on-the-nhl-25d16649cd52

#### Projet Live Feed (NHL API)
##### Utilise le live feed pour shot sur gardien
https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d

#### Docu NHL API
https://github.com/dword4/nhlapi

Ancienne Doc:
https://webcache.googleusercontent.com/search?q=cache:ILYmzDuxW70J:https://github.com/dword4/nhlapi+&cd=1&hl=en&ct=clnk&gl=ca

In [36]:
# Check Python Version
from platform import python_version
import pandas as pd
import requests  
from pandas.io.json import json_normalize
import datetime

pd.set_option('display.max_columns', None)

#print(pd.show_versions())

In [43]:
def extract_gamestats(gameID):
    
    '''
    This function takes the gameID as an input and returns a dataframe that contains
    the teamSkaterStats info for the home/away teams for that specific game.
    '''
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(gameID))
    data = r.json()
    df = pd.json_normalize(data = data['teams'])
    df = df.filter(regex='teamSkaterStats')
    df['gameID'] = gameID
    
    return df

### import game data

In [ ]:
start_date = datetime.date(2018, 4, 21)
end_date = datetime.date(2019, 4, 23)
delta = datetime.timedelta(days=1)

data_list = []

while start_date <= end_date:
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/schedule?date=' + start_date.strftime("%Y-%m-%d"))
    data = r.json()
    
    df = pd.json_normalize(data = data['dates'], record_path='games', meta=['date'])
    
    # On ajoute des statistiques au niveau des matchs
    # Le if sert a skipper les journees sans matchs
    if df.empty==False :
        pd_list = []
        for games in df['gamePk']:
            tmp = extract_gamestats(gameID=games)
            pd_list.append(tmp)

        pd_tmp = pd.concat(pd_list, sort='False', ignore_index=True)
        df = df.merge(pd_tmp, left_on='gamePk', right_on='gameID', how='left')

        data_list.append(df)
        
        start_date += delta
    else:
        start_date += delta

In [50]:
df = pd.concat(data_list, sort='False', ignore_index=True)
df.to_csv('../../data/games.csv')

### Import teams data

In [12]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'
r = requests.get(url)
json_data = r.json()
teams = json_normalize(json_data['teams'], sep = "_")

teams.to_csv('../../data/teams.csv')

C:\Users\athompson-leduc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.
